In [25]:
import os
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import (
    FilePurpose,
    FileSearchTool,
    ListSortOrder,
    RunAdditionalFieldList,
    RunStepFileSearchToolCall,
    RunStepToolCallDetails,
    ResponseFormat,
)
from azure.identity import DefaultAzureCredential

asset_file_path = os.path.abspath("./analyzer_output/financial_data.json")

project_client = AIProjectClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

agents_client = project_client.agents
# Upload file and create vector store
# [START upload_file_create_vector_store_and_agent_with_file_search_tool]
file = agents_client.files.upload_and_poll(file_path=asset_file_path, purpose=FilePurpose.AGENTS)
print(f"Uploaded file, file ID: {file.id}")

vector_store = agents_client.vector_stores.create_and_poll(file_ids=[file.id], name="financial_data")
print(f"Created vector store, vector store ID: {vector_store.id}")

# Create file search tool with resources followed by creating agent
file_search = FileSearchTool(vector_store_ids=[vector_store.id])

agent = agents_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="my-agent",
    instructions="""You are a financial data assistant that MUST ALWAYS use the file search tool to retrieve information from the financial_data.json file before providing any response.

    IMPORTANT: For EVERY user query, you MUST:
    1. ALWAYS call the file search tool first to search the financial_data.json file
    2. Extract the relevant financial data from the search results
    3. Format your response as JSON

    The output should be in the following format:
    {
        "company": "Tesco",
        "year": "2024",
        "financial_metrics": {
            "metric1": "value1",
            "metric2": "value2"
        }
    }

    NEVER provide information without first searching the file. If you cannot find relevant information in the file, return "No relevant information found".""",
    description="This agent is used to search information from financial data in json format, and return the information in json format. The financial data is in the file uploaded to the vector store, containg the financial data for the company Unilever and Tesco across years 2022-2024. The agent will return the information in json format.",
    temperature=0.0,
    tools=file_search.definitions,
    tool_resources=file_search.resources,
)

print(f"Created agent, ID: {agent.id}")

Uploaded file, file ID: assistant-CvGk6Dtoa9mBS2iXswx4SL
Created vector store, vector store ID: vs_CCFbjzZBKfQrrj9ksba3zC8y
Created agent, ID: asst_Q1mLoV8ZrRn0mbx06UsAWmif


## Sample Request

In [26]:
# Create thread for communication
thread = agents_client.threads.create()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
message = agents_client.messages.create(
thread_id=thread.id, role="user", content="Hello, send an email with the datetime and weather information in New York?"
)
print(f"Created message, ID: {message.id}")

# Create and process agent run in thread with tools
run = agents_client.runs.create(
    thread_id=thread.id,
    agent_id=agent.id
)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")


Created thread, ID: thread_4NBbCT4HL02fZx6woZnAtigO
Created message, ID: msg_RQa6P6jae05YWTuzGDFtqDPm
Run finished with status: RunStatus.QUEUED


In [24]:
for run_step in agents_client.run_steps.list(
    thread_id=thread.id, run_id=run.id, include=[RunAdditionalFieldList.FILE_SEARCH_CONTENTS]
):
    if isinstance(run_step.step_details, RunStepToolCallDetails):
        for tool_call in run_step.step_details.tool_calls:
            if (
                isinstance(tool_call, RunStepFileSearchToolCall)
                and tool_call.file_search
                and tool_call.file_search.results
                and tool_call.file_search.results[0].content
                and tool_call.file_search.results[0].content[0].text
            ):
                print(
                    "The search tool has found the next relevant content in "
                    f"the file {tool_call.file_search.results[0].file_name}:"
                )
                # Note: technically we may have several search results, however in our example
                # we only have one file, so we are taking the only result.
                print(tool_call.file_search.results[0].content[0].text)
                print("===============================================================")

# Fetch and log all messages
messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

# Print last messages from the thread
file_name = os.path.split(asset_file_path)[-1]
for msg in messages:
    if msg.text_messages:
        last_text = msg.text_messages[-1].text.value
        for annotation in msg.text_messages[-1].text.annotations:
            citation = (
                file_name if annotation.file_citation.file_id == file.id else annotation.file_citation.file_id
            )
            last_text = last_text.replace(annotation.text, f" [{citation}]")
        print(f"{msg.role}: {last_text}")

The search tool has found the next relevant content in the file financial_data.json:
{
            "Metric": "Total_Loans",
            "2022": null,
            "2023": null,
            "2024": null
        },
        {
            "Metric": "Leased_Space",
            "2022": null,
            "2023": null,
            "2024": null
        },
        {
            "Metric": "Total_Available_Space",
            "2022": null,
            "2023": null,
            "2024": null
        },
        {
            "Metric": "Average_Subscribers",
            "2022": null,
            "2023": null,
            "2024": null
        },
        {
            "Metric": "Total_Service_Revenue",
            "2022": null,
            "2023": null,
            "2024": null
        },
        {
            "Metric": "Subscribers_at_Start_End_of_Period",
            "2022": null,
            "2023": null,
            "2024": null
        },
        {
            "Metric": "Customers_at_Start_End_of_Pe

## Clean-up

In [ ]:
agents_client.vector_stores.delete(vector_store.id)
print("Deleted vector store")
agents_client.files.delete(file_id=file.id)
print("Deleted file")
agents_client.delete_agent(agent.id)
print("Deleted agent")